In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams['font.family'] = 'Times New Roman'
plt.rcParams['axes.unicode_minus'] = False
import warnings
# Ignore all warnings
warnings.filterwarnings("ignore")

path = r"database.xlsx"
df = pd.read_excel(path)
df

In [ ]:
from sklearn.model_selection import train_test_split
# Split features and target variables
X = df.drop([''], axis=1)  
y = df['']  

# Split the dataset into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X,  y,  test_size=0.2,  random_state=42)

In [ ]:
import xgboost as xgb
from sklearn.model_selection import GridSearchCV

# XGBoost parameters
params_xgb = {
    'booster': 'gbtree',              # Boosting method, here using Gradient Boosting Tree.
    'objective': 'reg:squarederror',  # Loss function, here using squared error, which is suitable for regression tasks.
    'max_leaves': 127,                # The number of leaf nodes per tree, which controls the model complexity.
    'verbosity': 1,                   # The verbosity of XGBoost's output. 0 indicates no output, while 1 indicates outputting progress information.
    'seed': 42,                       # Random seed, used to reproduce the model's results.
    'nthread': -1,                    # The number of threads for parallel computation, where -1 indicates using all available CPU cores.
    'colsample_bytree': 0.6,          # The proportion of features randomly selected for each tree, used to enhance the model's generalization ability.
    'subsample': 0.7,                 # The proportion of samples randomly selected in each iteration, used to enhance the model's generalization ability.
    'eval_metric': 'rmse'             # Evaluation metric, here using RMSE.
}


# Initialize the XGBoost classification model
model_xgb = xgb.XGBRegressor(**params_xgb)


# Define a parameter grid for grid search
param_grid = {
    'n_estimators': [100, 200, 300, 400, 500],  # Tree number
    'max_depth': [3, 4, 5, 6, 7],               # Tree depth
    'learning_rate': [0.01, 0.02, 0.05, 0.1],   # Learning rate
}


# Use GridSearchCV for grid search and k-fold cross-validation.
grid_search = GridSearchCV(
    estimator=model_xgb,
    param_grid=param_grid,
    scoring='neg_mean_squared_error',  # Evaluation metric is negative MSE.
    cv=10,                              # 10-fold cross-validation.
    n_jobs=-1,                         # Parallel computing.
    verbose=1                          # Output detailed progress information.
)

# Model training
grid_search.fit(X_train, y_train)

best_model_xgboost = grid_search.best_estimator_

In [ ]:
import shap
explainer = shap.TreeExplainer(best_model_xgboost)
shap_values = explainer.shap_values(X_test)

In [ ]:
shap_values_df = pd.DataFrame(shap_values, columns=X_test.columns)
shap_values_df.head()

In [ ]:
# Calculate the mean of the absolute values of each column
mean_abs_shap_values = shap_values_df.abs().mean()
shap_values_with_mean = shap_values_df.copy()  
shap_values_with_mean.columns = [f"{col} ({mean_abs_shap_values[col]:.3f})" for col in shap_values_with_mean.columns]
shap_values_with_mean.head()

In [ ]:
plt.figure()
shap.summary_plot(shap_values, X_test, feature_names=shap_values_with_mean.columns, plot_type="dot", show=False)
plt.savefig("Total effects.pdf", format='pdf',bbox_inches='tight',dpi=1200)
plt.show()

In [ ]:
# Calculate SHAP interaction values
# Extract the diagonal elements to represent the main effect values
shap_interaction_values = explainer.shap_interaction_values(X_test) 
main_effects = np.array([shap_interaction_values[:, i, i] for i in range(shap_interaction_values.shape[1])]).T
main_effects_df = pd.DataFrame(main_effects, columns=X_test.columns)
main_effects_df

In [ ]:
# Calculate the mean of the absolute values of each column
mean_abs_shap_vaMain_effects = main_effects_df.abs().mean()
shap_values_with_mean_vaMain_effects = main_effects_df.copy()  
shap_values_with_mean_vaMain_effects.columns = [f"{col} ({mean_abs_shap_vaMain_effects[col]:.3f})" for col in shap_values_with_mean_vaMain_effects.columns]
shap_values_with_mean_vaMain_effects.head()

In [ ]:
plt.figure()
shap.summary_plot(np.array(main_effects_df), X_test, feature_names=shap_values_with_mean_vaMain_effects.columns, plot_type="dot", show=False)
plt.xlabel('Main effect SHAP')
plt.savefig("vaMain effects.pdf", format='pdf',bbox_inches='tight',dpi=1200)
plt.show()

In [ ]:
# Get Feature Name
feature_names = X_test.columns

# Initialize an empty DataFrame, a matrix of size equal to the number of features
interaction_matrix = pd.DataFrame(np.nan, index=feature_names, columns=feature_names)

# Traversing each pair of features
for i, feature in enumerate(feature_names):
    for j, other_feature in enumerate(feature_names):
        if i != j:  # Consider only the interactions between different features
            # Calculate the absolute value of each sample's SHAP interaction value
            interaction_values = [shap_interaction_values[sample_idx][i, j] for sample_idx in range(shap_interaction_values.shape[0])]
            # Calculate the average absolute value of interaction values
            avg_interaction_value = np.mean(np.abs(interaction_values))
            # Assign the result to the corresponding matrix 
            interaction_matrix.loc[feature, other_feature] = avg_interaction_value
        else:
            # Calculate the main effect values (values on the diagonal)
            main_effect_values = [shap_interaction_values[sample_idx][i, i] for sample_idx in range(shap_interaction_values.shape[0])]
            # Calculate the average absolute value of main effect values
            avg_main_effect_value = np.mean(np.abs(main_effect_values))
            # Assign the main effect values to the diagonal positions
            interaction_matrix.loc[feature, feature] = avg_main_effect_value

interaction_matrix

In [ ]:
features = interaction_matrix.columns
main_effect_values = np.diag(interaction_matrix)
# Sort the main effect values from high to low
sorted_effects = sorted(zip(features, main_effect_values), key=lambda x: x[1], reverse=True)
# Extract the sorted feature names and main effect values
sorted_features, sorted_main_effect_values = zip(*sorted_effects)
# Create an empty matrix to store the interaction effect values
interaction_values_matrix = np.zeros((len(features), len(features)))

# Fill the interaction effect matrix, considering only non-diagonal elements (interaction effects)
for i, feature in enumerate(features):
    for j, other_feature in enumerate(features):
        if i != j:  # Consider only the interaction effects of off-diagonals
            interaction_values_matrix[i, j] = interaction_matrix.iloc[i, j]*2

bar_width = 0.35

index = np.arange(len(sorted_features))

plt.figure(figsize=(12, 6))

plt.bar(index - bar_width / 2, sorted_main_effect_values, bar_width, label="Main Effect", color="#FFFF00",alpha=0.8)

bars_data = interaction_values_matrix.T  # Transpose, for easier stacking.
bottom = np.zeros(len(features))  # Used as the base for stacking.

# Draw an interaction effect stacked bar chart (right)
for i in range(len(features)):
    # Plot each stacked component.
    bars = plt.bar(index + bar_width / 2, bars_data[i], bar_width, bottom=bottom, label=f'Interaction with {features[i]}')
    bottom += bars_data[i]  

# Display the total sum of the stack at the top of the stacked bar chart
for i in range(len(features)):
    total_height = np.sum(bars_data[:, i])  # Calculate the sum of each column
    if total_height > 0:
        plt.text(index[i] + bar_width / 2, bottom[i], f'{total_height:.3f}', ha='center', va='bottom', fontsize=13)

plt.xticks(index, sorted_features, ha='center')

plt.xlabel("")
plt.ylabel("Mean Absolute SHAP Values", fontsize=14)
plt.title("")

plt.tick_params(axis='both', which='major', labelsize=14)

for i, rect in enumerate(plt.gca().patches[:len(sorted_features)]):
    height = rect.get_height()
    plt.text(rect.get_x() + rect.get_width() / 2, height, f'{height:.3f}', ha='center', va='bottom', fontsize=13)

plt.legend(loc='upper center', bbox_to_anchor=(0.5, 1.3), ncol=3, fontsize=12)
plt.grid(axis='y', linestyle='--', alpha=0.6)
plt.tight_layout()
plt.savefig("Main and Interaction Effects.pdf", format='pdf',bbox_inches='tight',dpi=1200)
plt.show()